# Practice notebook for regression analysis with dependent data in NHANES

This notebook will give you the opportunity to perform some analyses
using the regression methods for dependent data that we are focusing
on in this week of the course.

Enter the code in response to each question in the boxes labeled "enter your code here".
Then enter your responses to the questions in the boxes labeled "Type
Markdown and Latex".

This notebook is based on the NHANES case study notebook for
regression with dependent data.  Most of the code that you will need
to write below is very similar to code that appears in the case study
notebook.  You will need to edit code from that notebook in small ways
to adapt it to the prompts below.

To get started, we will use the same module imports and read the data
in the same way as we did in the case study:

In [74]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import numpy as np

url = "nhanes_2015_2016.csv"
da0 = pd.read_csv(url)

vars = ["BPXSY1", "RIDAGEYR", "RIAGENDR", "RIDRETH1", "DMDEDUC2", "BMXBMI",
        "SMQ020", "SDMVSTRA", "SDMVPSU", 'BPXDI1']
da = da0[vars].dropna()

da["group"] = 10*da.SDMVSTRA + da.SDMVPSU

## Question 1: 

Build a marginal linear model using GEE for the first measurement of diastolic blood pressure (`BPXDI1`), accounting for the grouping variable defined above.  This initial model should have no covariates, and will be used to assess the ICC of this blood pressure measure.

In [72]:
formula = "BPXDI1 ~ 1"  
model = sm.GEE.from_formula(formula=formula, groups='group', cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
result.cov_struct.summary()

'The correlation between two observations in the same cluster is 0.031'

In [73]:
model = sm.GEE.from_formula("BPXSY1 ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.030


__Q1a.__ What is the ICC for diastolic blood pressure?  What can you
  conclude by comparing it to the ICC for systolic blood pressure?

The estimated ICC is 0.031, which is higher than ICC for sytolic blood pressure, 0.03

## Question 2: 

Take your model from question 1, and add gender, age, and BMI to it as covariates.

In [75]:
formula = "BPXDI1 ~ RIAGENDR + RIDAGEYR + BMXBMI"  
model = sm.GEE.from_formula(formula=formula, groups='group', cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
result.cov_struct.summary()

'The correlation between two observations in the same cluster is 0.030'

__Q2a.__ What is the ICC for this model?  What can you conclude by comparing it to the ICC for the model that you fit in question 1?

ICC now is 0.03, just slightly lower than for the Q1

## Question 3: 

Split the data into separate datasets for females and for males and fit two separate marginal linear models for diastolic blood pressure, one only for females, and one only for males.

In [76]:
# Create separate datasets for females and males
female_data = da[da['RIAGENDR'] == 2]  # Female gender code is 2
male_data = da[da['RIAGENDR'] == 1]    # Male gender code is 1

# Fit marginal linear models for diastolic blood pressure separately for females and males
formula = "BPXDI1 ~ 1"  # Model with no covariates

# For females
model_female = sm.GEE.from_formula(formula=formula, groups='group', cov_struct=sm.cov_struct.Exchangeable(), data=female_data)
result_female = model_female.fit()

# For males
model_male = sm.GEE.from_formula(formula=formula, groups='group', cov_struct=sm.cov_struct.Exchangeable(), data=male_data)
result_male = model_male.fit()

# Display summaries of the models
print("Summary for Female Model:")
print(result_female.summary())

print("\nSummary for Male Model:")
print(result_male.summary())


Summary for Female Model:
                               GEE Regression Results                              
Dep. Variable:                      BPXDI1   No. Observations:                 2640
Model:                                 GEE   No. clusters:                       30
Method:                        Generalized   Min. cluster size:                  59
                      Estimating Equations   Max. cluster size:                 125
Family:                           Gaussian   Mean cluster size:                88.0
Dependence structure:         Exchangeable   Num. iterations:                     6
Date:                     Thu, 31 Aug 2023   Scale:                         144.955
Covariance type:                    robust   Time:                         12:20:05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     68.7721      0.401    171.362      0.000      

__Q3a.__ What do you learn by comparing these two fitted models?

By comparing the summaries of the two fitted models for diastolic blood pressure, some key observations can be made:

1. Intercept (Baseline Diastolic Blood Pressure):
   - For females: The intercept (baseline diastolic blood pressure) is approximately 68.77 mmHg.
   - For males: The intercept (baseline diastolic blood pressure) is approximately 71.25 mmHg.
   - This suggests that, on average, males tend to have slightly higher diastolic blood pressure compared to females.

2. Skew and Kurtosis:
   - Skew and kurtosis values indicate the shape and distribution of the data.
   - For both female and male models, the skew values are negative, indicating a slightly left-skewed distribution.
   - The kurtosis values are within the range of normality for both models.

3. Cluster Sizes and Observations:
   - The number of observations in the female model is 2640, while in the male model, it's 2462.
   - The cluster sizes (mean number of observations per cluster) are 88.0 for females and 82.1 for males.

4. Conclusion:
   - Comparing the intercepts, it seems that males tend to have slightly higher baseline diastolic blood pressure than females.
   - The skew and kurtosis values suggest that the diastolic blood pressure distribution is slightly left-skewed, but overall well-behaved in both groups.
   - The difference in cluster sizes might impact the robustness of the estimates, as the clusters in the male model have slightly smaller sizes.


## Question 4: 

Using the entire data set, fit a multilevel model for diastolic blood pressure, predicted by age, gender, BMI, and educational attainment.  Include a random intercept for groups.

In [77]:
formula = "BPXDI1 ~ RIDAGEYR + RIAGENDR + BMXBMI + C(DMDEDUC2)"  
model = sm.MixedLM.from_formula(formula=formula, groups='group', data=da)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
            Mixed Linear Model Regression Results
==============================================================
Model:               MixedLM  Dependent Variable:  BPXDI1     
No. Observations:    5102     Method:              REML       
No. Groups:          30       Scale:               154.2055   
Min. group size:     106      Log-Likelihood:      -20115.7588
Max. group size:     226      Converged:           Yes        
Mean group size:     170.1                                    
--------------------------------------------------------------
                   Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
--------------------------------------------------------------
Intercept          70.201    1.234 56.872 0.000  67.782 72.621
C(DMDEDUC2)[T.2.0]  0.736    0.726  1.013 0.311  -0.688  2.159
C(DMDEDUC2)[T.3.0] -0.139    0.644 -0.216 0.829  -1.401  1.122
C(DMDEDUC2)[T.4.0]  0.527    0.618  0.852 0.394  -0.685  1.739
C(DMDEDUC2)[T.5.0]  0.955    0.641  1.489 0.137  -0.302  2.212
C(DMDEDUC2)[T.9.0] -2.978    8.823 -0.338 0.736 -20.272 14.315
RIDAGEYR           -0.039    0.010 -3.870 0.000  -0.059 -0.019
RIAGENDR           -2.756    0.351 -7.860 0.000  -3.444 -2.069
BMXBMI              0.186    0.026  7.286 0.000   0.136  0.236
group Var           4.171    0.108                            
==============================================================

"""

__Q4a.__ How would you describe the strength of the clustering in this analysis?

The strength of clustering in this analysis can be assessed by examining the variance of the "group" variable. In this case, the "group Var" value is 4.171, which represents the estimated variance of the random intercepts across the 30 groups. A larger value of the group variance suggests stronger clustering, indicating that there is substantial variability in diastolic blood pressure across different groups, beyond what can be explained by the individual-level variables in the model.

__Q4b:__ Include a random intercept for age, and describe your findings.

By including a random intercept for age, the model considers the variability in the intercepts (baseline diastolic blood pressure) across different age groups. However, based on the provided summary, it seems that the random intercept for age is not included in the model, as there is no information related to it. If you would like to include a random intercept for age, you would need to modify the model formula accordingly and then interpret the results to see how the clustering effect of age impacts the diastolic blood pressure.